# Check for Stationarity for each pair

In [22]:
import pandas as pd
import yfinance as yf
import random
import json
import statsmodels.api as sm
import datetime
import os 
import csv
import copy 
from statsmodels.tsa.stattools import adfuller

random.seed(1)


# Function: Load data obtained from the identify pairs notebook

In [23]:
def load_pairs():
    file_path = "data/pairs_names.json"
    
    # Read the JSON file as a dictionary
    with open(file_path, "r") as json_file:
        loaded_pairs_dic = json.load(json_file)
    
    # Now, loaded_pairs_dic contains the dictionary from the JSON file
    #print(loaded_pairs_dic)
    return loaded_pairs_dic



In [24]:
# Enable to use start & end dates:
start = datetime.date.today() - datetime.timedelta(days=430)
end = datetime.date.today() - datetime.timedelta(days=60)

# Function: Plotting price movement of two assets

In [25]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_assets(asset1, asset2, ticker1, ticker2):
    fig = make_subplots(rows=2,
                    cols=1,
                    subplot_titles=("asset2", "asset1"))

    fig.append_trace(go.Scatter(x=asset2.index,
                                y=asset2,
                                ),
                     row=1, col=1)
    
    fig.append_trace(go.Scatter(x=asset1.index,
                                y=asset1,
                                ),
                     row=2, col=1)
    
    # Update yaxis properties
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_xaxes(title_text="Date",  row=1, col=1)
    fig.update_yaxes(title_text="Price", row=2, col=1)
    fig.update_xaxes(title_text="Date",  row=2, col=1)
    
    #fig.show()
    if not os.path.exists("img/"):
        os.mkdir("img/")
        print("Image Directory Created!")
    
    file_path = "img/" + ticker1 + "_" + ticker2 + ".html" 
    fig.write_html(file_path)


# Function: Run OLS Regression and calculating spread
Since we are not using the recent 60 days data let us not split into training and testing

In [26]:
def regression_analysis(asset1, asset2, ticker1, ticker2):

    # adding a constant variable to the asset1 time series for the intercept value 
    asset1 = sm.add_constant(asset1, prepend=False)
    
    # Running the OLS function with asset2 as the dependent variable and asset1 as the dependent variable
    ols = sm.OLS(asset2, asset1)
    
    # Saving the results of the OLS model into a variable
    output = ols.fit()
    print("Completed Training")
    
    #print(output.params)
    # Getting the beta from the OLS results. Close means the close price of asset1
    beta = output.params["Close"]
    
    # Dropping the const column in the dependent variable
    asset1.drop(columns="const", inplace = True)
    
    # Calculating spread
    spread = asset2 - beta*asset1["Close"]
    
    fig = go.Figure(go.Scatter(y = spread))
    
    fig.update_layout(title = "Spread between " + ticker2 + " and " + ticker1)
    
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="Spread")
    
    file_path = "img/Spread_" + ticker1 + "_" + ticker2 + ".html"
    fig.write_html(file_path)
    
    #fig.show()
    
    return beta, spread
    


# Function: Perform Augmented Dickey Fuller Test

In [27]:
from statsmodels.tsa.stattools import adfuller
def run_adf_test(ticker1, ticker2, spread, pvalue_thres):
    result = adfuller(spread)
    does_pass = result[1] < pvalue_thres
    if does_pass:
        print(f'spread between {ticker1} and {ticker2} passed ADF stationarity test')
    else:
        print(f'spread between {ticker1} and {ticker2} failed ADF stationarity test')    
    return does_pass




1. p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
2. p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

**Since the p value is greater than 0.05 we fail to reject the null hypothesis, the time series is not stationary**

If it was accepted the code would be as follows
```python
accepted_pairs = []
accepted_pairs.append(target_pair[0] + "_" + target_pair[1]) 
```

You don't need to print out the values from ADF just check if the 5% level is less than or equal to 0.05 and save it into the list. 


Once the list is complete for all assets you would need to save it as a text file for later use



In [40]:

def get_data(ticker, start, end):
    return yf.download(ticker, start, end)['Close']

def main():
    final_pair_df = None
    
    adf_pvalue_thres = 0.05
    pair_to_beta = {} 
    
    pairs_dict = load_pairs()
    print(pairs_dict)

    for ticker1 in pairs_dict.keys():
        tickers = copy.copy(pairs_dict[ticker1])
        if isinstance(tickers, str):
            tickers = [tickers]
            
        for ticker2 in tickers:
            pair_name = ticker1 + '_' + ticker2
            print(pair_name)
            asset1 = get_data(ticker1,start,end)
            asset2 = get_data(ticker2,start,end)
            plot_assets(asset1, asset2, ticker1, ticker2)
            beta, spread = regression_analysis(asset1, asset2, ticker1, ticker2)
            pair_to_beta[pair_name] = beta
            does_pass_adf = run_adf_test(ticker1,ticker2,spread, adf_pvalue_thres)
            if does_pass_adf:
                data_row = pd.DataFrame(data={"Asset1":ticker1, "Asset2": ticker2, "Beta": beta}, index=[0])
                if final_pair_df is None:
                    final_pair_df = data_row
                else:
                    final_pair_df = pd.concat([final_pair_df, data_row],ignore_index=True)
                    
    print(final_pair_df)
    print(len(final_pair_df))
    final_pair_file_path = "data/final_pairs.csv"
    final_pair_df.to_csv(final_pair_file_path)
    '''
    with open(final_pair_file_path, "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        for pair in final_pair_list:
            tickers = pair.split("_")
            writer.writerow([tickers[0],tickers[1],pair_to_beta[pair]])
    '''
        
main()

{'ADYEY': ['ACN', 'ADSK'], 'AMAT': ['ADSK'], 'AMD': ['ADYEY'], 'APH': ['ADSK', 'ADYEY'], 'ASX': ['ANET'], 'BILL': ['ADSK'], 'BSY': ['ADSK', 'ADYEY', 'AMAT', 'AVGO'], 'CRM': ['ADSK', 'CDAY'], 'CRWD': ['ADSK', 'CDAY'], 'CSCO': ['ADI', 'APH', 'ASX', 'CDAY'], 'CTSH': ['ADSK'], 'DDOG': ['ADSK', 'APP', 'CDAY'], 'DELL': ['ACN', 'ADYEY', 'APH', 'CDAY'], 'DIDIY': ['CDAY'], 'DT': ['BSY'], 'ENPH': ['ANET', 'CDAY'], 'ENTG': ['ADSK', 'CDAY', 'CTSH'], 'ERIC': ['CDAY'], 'FI': ['CRM', 'ENPH'], 'FIS': ['ADSK', 'CDAY', 'ERIC'], 'FLEX': ['CDAY'], 'FSLR': ['ANET', 'ENPH', 'FIS'], 'FTNT': ['ADSK', 'ANET', 'FSLR'], 'FTV': ['ADBE', 'ADSK', 'ADYEY', 'APH', 'CTSH', 'DELL'], 'GDDY': ['ADSK'], 'GEN': ['CDAY', 'GDDY'], 'GFS': ['CDAY', 'GDDY'], 'GLW': ['ADSK', 'CDAY', 'GDDY', 'GFS'], 'GRAB': ['ADSK', 'CTSH', 'GDDY'], 'HPE': ['ADSK', 'APH', 'GDDY', 'GFS'], 'HPQ': ['ACN', 'ADSK', 'ADYEY', 'CTSH', 'GDDY', 'GFS'], 'IBM': ['ADSK', 'GDDY'], 'INFY': ['GDDY', 'GEN'], 'INTC': ['ACN', 'ADSK', 'ADYEY', 'APH', 'CDAY', 'GDDY',